### Pipelining Two Models
This notebook is an example of how to use DeGirum PySDK to do AI inference of a video file using two AI models: face detection and gender classification. The face detection model is run on the image and the results are then processed by the gender classification model, one face at a time. Combined result is then displayed.

In [ ]:
import degirum as dg, degirum_tools

inference_host_address = "@local"
zoo_url = 'degirum/hailo'
token=''
device_type=['HAILORT/HAILO8L']

# specify model names
face_det_model_name = "yolov8n_relu6_face--640x640_quant_hailort_hailo8l_1"
gender_cls_model_name = "yolov8n_relu6_fairface_gender--256x256_quant_hailort_hailo8l_1"

# specify video source
video_source = "../assets/faces_and_gender.mp4"

# Load face detection and gender detection models
face_det_model = dg.load_model(
    model_name=face_det_model_name,
    inference_host_address=inference_host_address,
    zoo_url=zoo_url,
    token='',
    device_type=device_type,
    overlay_color=[(255,255,0),(0,255,0)]    
)

gender_cls_model = dg.load_model(
    model_name=gender_cls_model_name,
    inference_host_address=inference_host_address,
    zoo_url=zoo_url,
    token='',
    device_type=device_type,
)

# Create a compound cropping model with 20% crop extent
crop_model = degirum_tools.CroppingAndClassifyingCompoundModel(
    face_det_model, 
    gender_cls_model, 
    30.0
)

# run AI inference on video stream
inference_results = degirum_tools.predict_stream(crop_model, video_source)

# display inference results
# Press 'x' or 'q' to stop
with degirum_tools.Display("Faces and Gender") as display:
    for inference_result in inference_results:
        display.show(inference_result.image_overlay)